# Data aggregation for visualization

* [Load data](#Load-data)
* [Aggregation by race](#Aggregation-by-race)
* [Aggregation by name](#Aggregation-by-name)

## Load data

In [1]:
import pandas as pd

In [2]:
# It's a bit long but you can load a remote CSV file from its URL !!! :fire:
raw_df = pd.read_csv('https://www.dropbox.com/s/tt9z5bik6uqndbz/full_database.csv?dl=1')

In [3]:
raw_df.shape

(1281195, 12)

In [4]:
raw_df.head()

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace,len_name
0,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abächerli Walter,M,1952,Hinwil,47,"4:31.56,1","0:53.5,3","0:6.26,0",2
1,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abbringh Ellen,F,1962,NL-Doorn 3941 EB,91,"5:55.9,4","2:12.11,6","0:8.25,0",2
2,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abegglen Eddy,M,1954,Mürren,424,"5:45.21,9","2:20.33,8","0:8.11,0",2
3,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.328675,Abosa Emebet,F,1974,Zuoz,1,"3:21.46,1",False,"0:4.46,0",2
4,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abplanalp Michel,M,1960,Auvernier,143,"4:30.26,9","1:7.35,3","0:6.24,0",2


## Extra info

In [5]:
races_info = pd.read_csv('../datasets/races-information.csv',index_col=0).drop('url', axis=1)
races_info.head()

,date,name,location,min_temp,max_temp,uv_index,weather_desc,latitude,longitude,weekday,day,month,year
0,sam. 27.03.1999,Männedörfler Waldlauf,Männedorf,NaN,NaN,NaN,NaN,47.2574625,8.6946733,saturday,27,3,1999
1,sam. 20.03.1999,Kerzerslauf,Kerzers,NaN,NaN,NaN,NaN,46.97488999999999,7.1954365,saturday,20,3,1999
2,sam. 24.04.1999,Luzerner Stadtlauf,Luzern,NaN,NaN,NaN,NaN,47.05016819999999,8.3093072,saturday,24,4,1999
3,sam. 24.04.1999,20km de Lausanne,Lausanne,NaN,NaN,NaN,NaN,46.5196535,6.6322734,saturday,24,4,1999
4,sam. 24.04.1999,"Chäsitzerlouf, Kehrsatz",Kehrsatz,NaN,NaN,NaN,NaN,NaN,NaN,saturday,24,4,1999


In [6]:
df = pd.merge(raw_df, races_info, how='left', left_on=['Race','Date'], right_on=['name','date']).drop(['date','name','location'],axis=1)
print(df.shape)
df.head()

(1281195, 22)


,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,...,min_temp,max_temp,uv_index,weather_desc,latitude,longitude,weekday,day,month,year
0,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abächerli Walter,M,1952,Hinwil,47,"4:31.56,1","0:53.5,3",...,NaN,NaN,NaN,NaN,NaN,NaN,saturday,6.0,9.0,2003.0
1,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abbringh Ellen,F,1962,NL-Doorn 3941 EB,91,"5:55.9,4","2:12.11,6",...,NaN,NaN,NaN,NaN,NaN,NaN,saturday,6.0,9.0,2003.0
2,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abegglen Eddy,M,1954,Mürren,424,"5:45.21,9","2:20.33,8",...,NaN,NaN,NaN,NaN,NaN,NaN,saturday,6.0,9.0,2003.0
3,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.328675,Abosa Emebet,F,1974,Zuoz,1,"3:21.46,1",False,...,NaN,NaN,NaN,NaN,NaN,NaN,saturday,6.0,9.0,2003.0
4,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195000,Abplanalp Michel,M,1960,Auvernier,143,"4:30.26,9","1:7.35,3",...,NaN,NaN,NaN,NaN,NaN,NaN,saturday,6.0,9.0,2003.0


## Convert some stuff

In [19]:
df.latitude = df.latitude.apply(float)
df.longitude = df.longitude.apply(float)

## Clean race names

In [20]:
def clean_name(x):
    return x.replace("/"," ").replace("\\"," ")

df.Race = df.Race.apply(clean_name)

## Convert time string to seconds (ignore further precision)

In [21]:
def time_to_seconds(x):
    # Get hours
    split = x.split(':')
    hours = int(split[0])
    # Get minutes
    split = split[1].split('.')
    minutes = int(split[0])
    # Get seconds
    split = split[1].split(',')
    seconds = int(split[0])
    return hours*3600 + minutes*60 + seconds

def seconds_to_time(x):
    minutes, seconds = divmod(x, 60)
    hours, minutes = divmod(minutes, 60)
    return '{}:{}:{}'.format(hours, minutes, seconds)

In [22]:
df['time'] = df.Time.apply(time_to_seconds)

## Add a race+date index

In [23]:
df['race_index'] = df.Race + ' ; ' + df.Date

## Final global dataframe

In [24]:
df.tail()

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,...,uv_index,weather_desc,latitude,longitude,weekday,day,month,year,time,race_index
1281190,"Course de l'Escalade, Genève",sam. 05.12.2015,7.334317,Zwahlen Guy,M,1959,Genève,920,"0:42.25,8","0:17.0,2",...,0.0,Sunny,46.204391,6.143158,saturday,5.0,12.0,2015.0,2545,"Course de l'Escalade, Genève ; sam. 05.12.2015"
1281191,"Course de l'Escalade, Genève",sam. 05.12.2015,7.323638,Zwahlen Laurent,M,1967,Concise,1417,"0:37.43,4","0:13.27,1",...,0.0,Sunny,46.204391,6.143158,saturday,5.0,12.0,2015.0,2263,"Course de l'Escalade, Genève ; sam. 05.12.2015"
1281192,"Course de l'Escalade, Genève",sam. 05.12.2015,7.325498,Zweigart Benjamin,M,1991,Confignon,357,"0:31.8,2","0:7.20,9",...,0.0,Sunny,46.204391,6.143158,saturday,5.0,12.0,2015.0,1868,"Course de l'Escalade, Genève ; sam. 05.12.2015"
1281193,"Course de l'Escalade, Genève",sam. 05.12.2015,7.342394,Zwicky Pierre,M,1963,Genève,1089,"0:45.2,1","0:18.21,8",...,0.0,Sunny,46.204391,6.143158,saturday,5.0,12.0,2015.0,2702,"Course de l'Escalade, Genève ; sam. 05.12.2015"
1281194,"Course de l'Escalade, Genève",sam. 05.12.2015,2.340375,Zwimpfer Maxime,M,2005,Vandoeuvres,223,"0:11.7,7","0:2.52,1",...,0.0,Sunny,46.204391,6.143158,saturday,5.0,12.0,2015.0,667,"Course de l'Escalade, Genève ; sam. 05.12.2015"


# Aggregation by race

In [25]:
df.columns

Index(['Race', 'Date', 'Distance', 'Name', 'Sex', 'Year', 'LivingPlace',
       'Rank', 'Time', 'Delay', 'Pace', 'len_name', 'min_temp', 'max_temp',
       'uv_index', 'weather_desc', 'latitude', 'longitude', 'weekday', 'day',
       'month', 'year', 'time', 'race_index'],
      dtype='object')

In [26]:
def keep_first(x):
    return x.unique()[0]

def str_count(x, s=''):
    try:
        count = x.value_counts()[s]
    except KeyError:
        count = 0
    return count

aggregations = {
    'Race': {'race': keep_first},
    'Date': {'date': keep_first},
    'Distance': 'max',
    'Name': 'count',
    'Sex': {
        'M': lambda x: str_count(x, s='M'),
        'F': lambda x: str_count(x, s='F')
    },
    'Year': {
        'min_year': 'min', 
        'max_year': 'max',
        'mean_year': 'mean',
        'median_year': 'median'
    },
    'time': {
        'min_time': lambda x: seconds_to_time(x.min()),
        'max_time': lambda x: seconds_to_time(x.max()),
        'mean_time': lambda x: seconds_to_time(x.mean()),
        'median_time': lambda x: seconds_to_time(x.median()),
    },
    'latitude': {'latitude': keep_first},
    'longitude': {'longitude': keep_first},
    'weather_desc': {'weather_desc': keep_first},
    'min_temp': {'min_temp': keep_first},
    'max_temp': {'max_temp': keep_first},
}

# ---- INTERLUDE ----
#
# Not sure what's best here :
# - groupby 'race_index' and export all data in one huge JSON file (choice 1)
# - groupby ['Race', 'Date'] and export one JSON file by race name (choice 2)
#
# Maybe choice 2 is more practical when working with D3 / viz tools...
# -------------------

races_stats = df.groupby('race_index').agg(aggregations)     # choice 1
# races_stats = df.groupby(['Race', 'Date']).agg(aggregations) # choice 2

races_stats.columns = races_stats.columns.droplevel(0)

In [27]:
races_stats.head()

,median_time,min_time,max_time,mean_time,F,M,race,date,max,latitude,min_year,max_year,mean_year,median_year,min_temp,max_temp,weather_desc,longitude,count
race_index,,,,,,,,,,,,,,,,,,,
10km Schweizer Meisterschaften Lyss ; sam. 02.04.2011,0.0:42.0:29.0,0:22:15,1:15:56,0.0:43.0:42.61190965092419,130,357,10km Schweizer Meisterschaften Lyss,sam. 02.04.2011,10.0,47.074650,1923,1997,1966.012320,1965.0,6.0,25.0,Clear,7.307702,487
10km Schweizer Meisterschaften Lyss ; sam. 31.03.2012,0.0:41.0:24.0,0:17:23,1:15:57,0.0:42.0:26.39381443298953,115,370,10km Schweizer Meisterschaften Lyss,sam. 31.03.2012,10.0,47.074650,1934,1998,1966.777320,1966.0,4.0,15.0,Cloudy,7.307702,485
10km de Payerne ; sam. 27.03.2010,0.0:38.0:58.0,0:1:23,1:28:14,0.0:35.0:24.844331641285862,188,403,10km de Payerne,sam. 27.03.2010,10.0,46.822027,1923,2007,1974.335025,1971.0,0.0,10.0,Patchy rain possible,6.940566,591
10km de Payerne ; sam. 28.03.2009,0.0:41.0:18.0,0:29:55,1:27:13,0.0:43.0:15.14673913043498,80,288,10km de Payerne,sam. 28.03.2009,10.0,46.822027,1923,1995,1964.978261,1966.0,2.0,5.0,Partly cloudy,6.940566,368
"10km, Payerne ; sam. 10.03.2012",0.0:42.0:50.0,0:1:26,1:14:23,0.0:38.0:23.193430656934197,140,408,"10km, Payerne",sam. 10.03.2012,10.0,46.822027,1943,2010,1976.005474,1973.0,-5.0,8.0,Sunny,6.940566,548


In [28]:
# Example of JSON export
races_stats[:2].to_json(orient='index')

'{"10km Schweizer Meisterschaften Lyss ; sam. 02.04.2011":{"median_time":"0.0:42.0:29.0","min_time":"0:22:15","max_time":"1:15:56","mean_time":"0.0:43.0:42.61190965092419","F":130,"M":357,"race":"10km Schweizer Meisterschaften Lyss","date":"sam. 02.04.2011","max":10.0,"latitude":47.0746504,"min_year":1923,"max_year":1997,"mean_year":1966.0123203285,"median_year":1965.0,"min_temp":6.0,"max_temp":25.0,"weather_desc":"Clear","longitude":7.3077022,"count":487},"10km Schweizer Meisterschaften Lyss ; sam. 31.03.2012":{"median_time":"0.0:41.0:24.0","min_time":"0:17:23","max_time":"1:15:57","mean_time":"0.0:42.0:26.39381443298953","F":115,"M":370,"race":"10km Schweizer Meisterschaften Lyss","date":"sam. 31.03.2012","max":10.0,"latitude":47.0746504,"min_year":1934,"max_year":1998,"mean_year":1966.7773195876,"median_year":1966.0,"min_temp":4.0,"max_temp":15.0,"weather_desc":"Cloudy","longitude":7.3077022,"count":485}}'

In [30]:
races_stats.to_json('races.json', orient='index')

**SOME COMMENTS FOR LATER**

* When aggregating, we should compute more detailed things, eg. time/gender, etc... to have something nice to visualize on the details page using D3.js

# Aggregation by name

**TO DO ! Beware of the name 'real' duplicates (two different persons with same name (and sometimes same birthyear...)), and the 'false' duplicates (different formatting for the same name, eg. trailing space).**

In [12]:
df.Name.value_counts().head()

Schmid Christian     180
Meier Thomas         170
Müller Martin        160
Müller Thomas        155
Meier Andreas        136
Name: Name, dtype: int64

In [13]:
name_agg_df = df[df.Name.str.strip() == 'Schmid Christian']

In [14]:
name_agg_df.head()

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace,len_name,time,race_index
2304,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195,Schmid Christian,M,1981,DE-Waldbronn,332,"4:38.55,9","1:49.54,3","0:6.36,0",2,16735,"Jungfrau-Marathon, Interlaken ; sam. 06.09.2003"
2305,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195,Schmid Christian,M,1967,Berikon,160,"4:12.29,0","1:23.27,4","0:5.59,0",2,15149,"Jungfrau-Marathon, Interlaken ; sam. 06.09.2003"
2306,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195,Schmid Christian,M,1963,Morrens VD,397,"5:15.51,2","1:52.59,6","0:7.29,0",2,18951,"Jungfrau-Marathon, Interlaken ; sam. 06.09.2003"
5569,"Jungfrau-Marathon, Interlaken",sam. 11.09.2004,42.195,Schmid Christian,M,1967,Berikon,183,"4:16.34,4","1:17.3,5","0:6.4,0",2,15394,"Jungfrau-Marathon, Interlaken ; sam. 11.09.2004"
9147,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,42.195,Schmid Christian,M,1968,Zürich,270,"4:29.35,3","1:30.13,5","0:6.23,0",2,16175,"Jungfrau-Marathon, Interlaken ; sam. 10.09.2005"
